<a href="https://colab.research.google.com/github/suirindo/awabi/blob/master/awabiVer_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Googleドライブから必要データのインポート
import google.colab
import googleapiclient.discovery
import googleapiclient.http

In [0]:
#colabからgoogleDriveへの認証と接続
google.colab.auth.authenticate_user()
drive_service = googleapiclient.discovery.build('drive', 'v3')

In [0]:
#colabへのファイルの取り込み
upload_filename = 'awabi.zip'

file_list = drive_service.files().list(q="name='" + upload_filename + "'").execute().get('files')

#ファイルIDの取得
file_ID = None
for file in file_list:
  if file.get('name') == upload_filename:
    file_id = file.get('id')
    break

if file_id is None:
  #ファイルIDを取得できなかった場合はエラーメッセージを出力
  print(upload_filename + ' が見つかりません')
else:
  # colab環境へファイルをアップロードする
  with open(upload_filename, 'wb') as f:
    request = drive_service.files().get_media(fileId=file_id)
    media = googleapiclient.http.MediaIoBaseDownload(f, request)
    
    done = False
    while not done:
      progress_status, done = media.next_chunk()
      print(100*progress_status.progress(), end="")
      print("%完了")

print('GoogleドライブからColab環境へのファイル取り込みが完了')


100.0%完了
GoogleドライブからColab環境へのファイル取り込みが完了


In [0]:
!unzip awabi.zip

Archive:  awabi.zip
replace awabi/sample_submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: awabi/sample_submit.csv  
replace awabi/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: awabi/test.csv          
  inflating: awabi/train.csv         


In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR # 線形回帰のモデル
%matplotlib inline

train = pd.read_csv('./awabi/train.csv') # 学習データの読み込み
test = pd.read_csv('./awabi/test.csv') # 検証データの読み込み
sample = pd.read_csv('./awabi/sample_submit.csv', header=None) # 提出用サンプルデータの読み込み

# データ確認
train.head()

,id,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0,M,0.530,0.435,0.155,0.6990,0.2880,0.1595,0.205,10
1,3,F,0.710,0.565,0.195,1.8170,0.7850,0.4920,0.490,11
2,7,F,0.625,0.520,0.180,1.3540,0.4845,0.3510,0.375,11
3,12,M,0.375,0.280,0.095,0.2225,0.0875,0.0430,0.080,10
4,16,I,0.415,0.330,0.090,0.3595,0.1700,0.0810,0.090,6


In [0]:
X_train = train[["Length", "Diameter", "Height","Whole weight","Shucked weight","Viscera weight","Shell weight"]]

In [0]:
y_train = train['Rings']

In [0]:
model = LR()

In [0]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#検証用データ
X_test = test[["Length", "Diameter", "Height","Whole weight","Shucked weight","Viscera weight","Shell weight"]]

#予測
pred = model.predict(X_test)
print(pred)

[ 9.42675638 11.13063454 16.5527267  ... 13.14092135 12.45324729
 11.92558643]


In [0]:
sample[1] = pred
sample.to_csv('submit1.csv', index = None, header = None)

In [0]:
from google.colab import files

#with open('my_tree_one.csv', 'w') as f:
#f.write('some content')
  
files.download('submit1.csv')  